In [125]:
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import os
import pickle

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [126]:
os.listdir() 

['Approach 3.ipynb',
 '.DS_Store',
 'deap dataset.ipynb',
 'Approach 1.ipynb',
 'Approach 4.ipynb',
 'Approach 2-Copy2.ipynb',
 'Emotion analysis implementation.ipynb',
 'Approach 2.ipynb',
 '.ipynb_checkpoints',
 'HCI-Emotion-analysis',
 'data_preprocessed_python',
 'FinalScript.ipynb']

1. We will take into consideration only one video 
2. Loop over all files 
3. for each file --> readFile
4. Convert to dataframe
5. Extract only data for the first element from data 
6. Extract labels data
6. Make average data 
7. Put in data dataFrame

8. Extract data as csv

In [158]:
# public general variables
fileNamesArr = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', ]

all_channels = np.array(['b1','b2', 'b3', 'b4', 'b5', 'b6','b7', 'b8', 'b9', 'b10','b11','b12', 'b13', 'b14', 'b15', 'b16','b17', 'b18', 'b19', 'b20','b21','b22', 'b23', 'b24', 'b25', 'b26','b27', 'b28', 'b29', 'b30', 'b31', 'b32', 'hEOG', 'vEOG', 'zEMG', 'tEMG', 'GSR', 'Respiration belt', 'Plethysmograph', 'Temp'])
EEG_data = np.array(['b1','b2', 'b3', 'b4', 'b5', 'b6','b7', 'b8', 'b9', 'b10','b11','b12', 'b13', 'b14', 'b15', 'b16','b17', 'b18', 'b19', 'b20','b21','b22', 'b23', 'b24', 'b25', 'b26','b27', 'b28', 'b29', 'b30', 'b31', 'b32'])
peripheral_channels = np.array(["zEMG", "tEMG", "GSR", "Respiration belt", "Plethysmograph", "Temperature"])

finalData = pd.DataFrame(columns = ["subjectID", "valence", "arousal", "dominance", "zEMG", "tEMG", "GSR", "Respiration belt", "Plethysmograph", "Temperature"])


In [159]:
# Reading the files where the data is in shape of 40 * 40 * 8064

def readFile(fileNumber):
    infile = open('data_preprocessed_python/s' + fileNumber + '.dat', 'rb')
    new_dict = pickle.load(infile, encoding="latin1")
    infile.close()
    labels=[]
    data=[]

    for keys in new_dict:
        if(keys=='labels'):
            labels=new_dict[keys]
        else:
            data=new_dict[keys]
            
#     print(len(data))
#     print(len(data[0])) 
#     print(len(data[0][0])) 
    return labels, data

#####################################################################################################################
def convertToDataframe(data):
    # drop EEG data 
    data = data[34:, :]
    
    df_data = pd.DataFrame(columns = peripheral_channels)
    df_dataPerVideo = pd.DataFrame()

    for i in range(0, len(data)): #video 40
        signal = data[i]
        df_dataPerVideo[peripheral_channels[i]] = signal
    
    return df_dataPerVideo

def averageData(data):
    averageData = []
    for i in range(0, len(peripheral_channels)):
        averageData.append(data[peripheral_channels[i]].mean())
    return averageData

#####################################################################################################################
def getMedians(labels_separated):
    valenceMedian = labels_separated['valence'].median()
    arousalMedian = labels_separated['arousal'].median()
    dominanceMedian = labels_separated['dominance'].median()
    
    return valenceMedian, arousalMedian, dominanceMedian

#####################################################################################################################
def getLabels(labels, valenceMedian, arousalMedian, dominanceMedian):
    valence = labelsArr[:,0]
    arousal = labelsArr[:,1]
    dominance = labelsArr[:,2]
    liking = labelsArr[:,3]

    labels_separated = pd.DataFrame({'valence': valence, 'arousal': arousal, 'dominance': dominance, 'liking': liking })

    valenceMedian, arousalMedian, dominanceMedian = getMedians(labels_separated)
    
    labelledData = []
    labelledData.append("pos" if labels_separated.iloc[0]['valence'] >= valenceMedian else "neg")
    labelledData.append("high" if labels_separated.iloc[0]['arousal'] >= arousalMedian else "low")
    labelledData.append("strong" if labels_separated.iloc[0]['dominance'] >= dominanceMedian else "weak")
    
    return labelledData

#####################################################################################################################
def concatAllData(df_fileNames, df_labelsEncoded, df_avgData):
    df_fileNames = pd.DataFrame(columns = ["subjectID"])
    df_avgData = pd.DataFrame(columns = peripheral_channels)
    df_labelsEncoded = pd.DataFrame(columns = ["valence", "arousal", "dominance"])
    concatedData = pd.DataFrame(columns = ["subjectID", "valence", "arousal", "dominance", "zEMG", "tEMG", "GSR", "Respiration belt", "Plethysmograph", "Temperature"])

    df_length = len(df_fileNames)
    df_fileNames.loc[df_length] = fileNumber

    df_length = len(df_labelsEncoded)
    df_labelsEncoded.loc[df_length] = labelsEncoded

    df_length = len(df_avgData)
    df_avgData.loc[df_length] = avgData

    concatedData = pd.concat([df_fileNames, df_labelsEncoded, df_avgData], axis=1)
    return concatedData


In [160]:
for i in range(0, len(fileNamesArr)):
    print(i)
    fileNumber = fileNamesArr[i]
    labelsArr, dataArr = readFile(fileNumber)
    data_df = convertToDataframe(dataArr[0]) #only data for the first video
    avgData = averageData(data_df)
    labelsEncoded = getLabels(labelsArr, valenceMedian, arousalMedian, dominanceMedian)
    
    concatedData = concatAllData(df_fileNames, df_labelsEncoded, df_avgData)
    finalData = pd.concat([finalData, concatedData], ignore_index=True,)
finalData

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31


,subjectID,valence,arousal,dominance,zEMG,tEMG,GSR,Respiration belt,Plethysmograph,Temperature
0,01,pos,high,strong,19.525680,-15.428396,1716.504173,441.855164,-209.929106,-0.042336
1,02,pos,low,strong,-66.011643,17.156563,69.741864,-650.437623,5905.735231,-0.051039
2,03,pos,high,strong,53.182453,441.431759,-1297.200318,-10.006841,-1470.215022,0.043477
3,04,pos,high,strong,-768.391375,139.977389,1332.809991,-1285.250047,3737.062783,-0.125947
4,05,pos,low,weak,251.818045,17.871483,13209.094796,4.196936,366.836240,0.189098
5,06,pos,low,strong,-54.224250,-19.380688,-3074.399430,-350.922304,-777.481433,-0.240969
6,07,pos,low,weak,43.378857,-78.164317,6787.379280,-2579.455997,45865.448233,-0.189884
7,08,pos,high,strong,-82.090486,60.115809,22129.531844,-2601.512934,-1960.601495,0.158360
8,09,pos,low,weak,220.177366,29.511428,-28240.368705,-2206.982839,-4952.442099,0.019038
9,10,pos,high,strong,70.860566,132.541023,1091.992425,-1277.562360,-1326.315142,0.018611
